# Brief 3

In [40]:
import pymongo
import feedparser

NewsFeed = feedparser.parse("http://rss.jobsearch.monster.com/rssquery.ashx?q=big_data")
entry = NewsFeed.entries[1]

print (entry.keys())

dict_keys(['title', 'title_detail', 'summary', 'summary_detail', 'links', 'link', 'id', 'guidislink', 'published', 'published_parsed'])


In [45]:
import feedparser

rss_url = "http://rss.jobsearch.monster.com/rssquery.ashx?q=big"
parser = feedparser.parse(rss_url)
#parser.keys()

for entry in parser.entries:
    print(entry.title)
    print(entry.links[0].href)
    print('------------------------------------------------------------------------------------------------')

Senior Big Data Engineer
http://jobview.monster.com/Senior-Big-Data-Engineer-Job-Richmond-VA-US-221987088.aspx
------------------------------------------------------------------------------------------------
Data Engineering Manager - Big Data
http://jobview.monster.com/Data-Engineering-Manager-Big-Data-Job-San-Bernardino-CA-US-221982331.aspx
------------------------------------------------------------------------------------------------
Big Data Developer
http://jobview.monster.com/Big-Data-Developer-Job-Tampa-FL-US-221978899.aspx
------------------------------------------------------------------------------------------------
Industrial Worker
http://jobview.monster.com/Industrial-Worker-Job-Big-Flats-NY-US-221587757.aspx
------------------------------------------------------------------------------------------------
Big Data Developer - II
http://jobview.monster.com/Big-Data-Developer-II-Job-Garden-City-NY-US-221152772.aspx
------------------------------------------------------------

In [81]:
import feedparser

#rss_url = "http://rss.jobsearch.monster.com/rssquery.ashx?q=big_data"
rss_url = "rssquery.xml"
feed = feedparser.parse(rss_url)
entry = feed.entries[0]

for entry in parser.entries:
    print (entry.title)
    print ("------------------------")
    print (entry.published)
    print ("******")
    print (entry.summary)
    print ("------ News Link --------")
    print (entry.link)

Senior Big Data Engineer
------------------------
Thu, 12 Nov 2020 07:37:30 GMT
******
VA-Richmond, Job Description We have an exciting opportunity for a Senior Big Data Engineer with our industry-leading client in Richmond, VA. We will accept corp to corp or w2 contractors. For our w2 consultants, we offer a great benefits package that includes Medical, Dental, and Vision benefits, 401k with company matching, and life insurance. Responsibilities: Build data pipeline frameworks to automate high-
------ News Link --------
http://jobview.monster.com/Senior-Big-Data-Engineer-Job-Richmond-VA-US-221987088.aspx
Data Engineering Manager - Big Data
------------------------
Thu, 12 Nov 2020 05:10:14 GMT
******
CA-San Bernardino, If you are a Data Engineering Manager with experience, please read on! Top Reasons to Work with Us Fortune 500 Company! What You Will Be Doing - Lead and Manage small to large IT project initiatives related to Data and Data Delivery. - Lead and Manage production applica

In [60]:
import scrapy
 
class ScrapeRssSpider(scrapy.Spider):
    name = 'scrape-rss'
    allowed_domains = ['https://www.blog.google/rss']
    start_urls = ['http://https://www.blog.google/rss/']
 
    def start_requests(self):
        urls = ['https://www.blog.google/rss',        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
 
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }

In [98]:
import scrapy

from scrapy.spiders import XMLFeedSpider

class Spider(XMLFeedSpider):
       name = "BigDataScrap"
       allowed_domains = ["http://rss.jobsearch.monster.com"]
       start_urls = ['http://rss.jobsearch.monster.com/rssquery.ashx?q=big_data']    #Crawl BPMX
       itertag = 'item'

       def parse_node(self, response, node):
           self.logger.info('Hi, this is a <%s> node!: %s', self.itertag, ''.join(node.extract()))

           item = {}
           item['title'] = node.xpath('title/text()',).extract_first()                #define XPath for title
           item['link'] = node.xpath('link/text()').extract_first()
           item['pubDate'] = node.xpath('link/pubDate/text()').extract_first()
           item['description'] = node.xpath('description/text()').extract_first()                #define XPath for description
           return item

In [97]:
parse_node

NameError: name 'parse_node' is not defined

In [1]:
import scrapy
from scrapy import Request
from pprint import pprint

class FlashbotSpider(scrapy.Spider):
    name = 'flashbot'
    allowed_domains = ['rss.jobsearch.monster.com']

    # Start the crawler at this URLs
    start_urls = ['file:///rssquery.xml']
    #start_urls = ['http://rss.jobsearch.monster.com/rssquery.ashx?q={query}']

    thesaurus = ["machine learning", "machine", "learning", "big data", "big", "data"]

    LOG_LEVEL = "INFO"

    def parse(self, response):

        # We stat with this url
        url = self.start_urls[0]

        # Build and send a request for each word of the thesaurus
        for query in self.thesaurus:
            target = url.format(query=query)
            print("fetching the URL: %s" % target)
            if target.startswith("file://"):
                r = Request(target, callback=self.scrapit, dont_filter=True)
            else:
                r = Request(target, callback=self.scrapit)
            r.meta['query'] = query
            yield r

    def scrapit(self, response):
        query = response.meta["query"]

        # Base item with query used to this response
        item = {"query": query}
        print(query, response)

        # Scrap the data
        for doc in response.xpath("//item"):
            item["title"] = doc.xpath("title/text()").extract()
            item["description"] = doc.xpath("description/text()").extract()
            item["link"] = doc.xpath("link/text()").extract()
            item["pubDate"] = doc.xpath("pubDate/text()").extract()
            item["guid"] = doc.xpath("guid/text()").extract()
            #pprint(item, indent=2)
            print("item scraped:", item["title"])
            yield item